## Why Long Polling?

When you need to stream data from a backend to a frontend but can't use WebSockets (e.g., certain deployment constraints, simpler client implementation), **long polling** is a reliable alternative.

The pattern:
1. Client makes a request
2. Server holds the connection open until data is available
3. Server sends response, client immediately reconnects
4. Repeat until streaming is complete

## Server Implementation

In [ ]:
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
import asyncio
from typing import AsyncGenerator

app = FastAPI()

async def generate_stream() -> AsyncGenerator[str, None]:
    """Simulate streaming data (e.g., LLM tokens)."""
    chunks = ["Hello ", "world! ", "This ", "is ", "streaming ", "data."]
    for chunk in chunks:
        await asyncio.sleep(0.3)  # Simulate delay
        yield chunk

@app.get("/stream")
async def stream_endpoint():
    return StreamingResponse(
        generate_stream(),
        media_type="text/plain"
    )

## Long Polling with Session State

For more complex scenarios where you need to track progress across requests:

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uuid
import asyncio
from collections import defaultdict

app = FastAPI()

# In-memory session storage (use Redis in production)
sessions: dict[str, dict] = {}

class StartResponse(BaseModel):
    session_id: str

class PollResponse(BaseModel):
    data: str
    done: bool

@app.post("/start")
async def start_stream() -> StartResponse:
    """Initialize a streaming session."""
    session_id = str(uuid.uuid4())
    sessions[session_id] = {
        "queue": asyncio.Queue(),
        "done": False
    }
    # Start background task to produce data
    asyncio.create_task(produce_data(session_id))
    return StartResponse(session_id=session_id)

async def produce_data(session_id: str):
    """Background task that produces streaming data."""
    chunks = ["Processing ", "your ", "request ", "now..."]
    for chunk in chunks:
        await asyncio.sleep(0.5)
        await sessions[session_id]["queue"].put(chunk)
    sessions[session_id]["done"] = True

@app.get("/poll/{session_id}")
async def poll(session_id: str) -> PollResponse:
    """Long poll for next chunk of data."""
    if session_id not in sessions:
        raise HTTPException(status_code=404, detail="Session not found")
    
    session = sessions[session_id]
    
    try:
        # Wait up to 30s for data
        data = await asyncio.wait_for(
            session["queue"].get(),
            timeout=30.0
        )
        return PollResponse(data=data, done=False)
    except asyncio.TimeoutError:
        # Return empty response, client should retry
        return PollResponse(data="", done=session["done"])

## Client Implementation (JavaScript)

In [ ]:
%%javascript
async function streamWithLongPolling() {
    // Start session
    const startRes = await fetch('/start', { method: 'POST' });
    const { session_id } = await startRes.json();
    
    // Poll until done
    let done = false;
    while (!done) {
        const pollRes = await fetch(`/poll/${session_id}`);
        const result = await pollRes.json();
        
        if (result.data) {
            console.log('Received:', result.data);
            // Update UI with streamed data
        }
        done = result.done;
    }
    console.log('Stream complete');
}

## Key Considerations

| Aspect | Recommendation |
|--------|----------------|
| Timeout | Set reasonable poll timeout (30s typical) |
| Session cleanup | Add TTL-based cleanup for abandoned sessions |
| Production storage | Use Redis instead of in-memory dict |
| Error handling | Client should retry on network errors |
| Scaling | Session affinity or shared state needed for multiple workers |

## When to Use What

- **SSE (Server-Sent Events)**: Best for one-way server-to-client streaming
- **WebSockets**: Bidirectional, real-time communication
- **Long Polling**: When SSE/WebSockets aren't available, or for simpler deployments